<a href="https://colab.research.google.com/github/patelkanak23/Image-Captioning/blob/main/Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Importing Required Libraries***


In [ ]:
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import os
import string
import re
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, add
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from nltk.translate.bleu_score import corpus_bleu
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import load_model

## ***Extracting Features from images***

In [ ]:
def extract_features(directory):


    features = dict()
    for name in listdir(directory):
        filename = directory + '/' + name
        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id] = feature
        print('>%s' % name)

    return features


In [ ]:
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
print(model.summary())

553467096/553467096 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [ ]:
directory = r"C:\ML-DATA\Flickr8k_Dataset\Flicker8k_Dataset"
features = extract_features(directory)
print('Extracted Features: %d' % len(features))

dump(features, open('features.pkl', 'wb'))

NameError: name 'listdir' is not defined

In [ ]:
from pickle import load
all_features = load(open('features.pkl', 'rb'))
len(list(all_features.values()))

8091

## ***Preparing Text Data***

In [ ]:
def load(file):
 file = open(filename, 'r')
 text = file.read()
 file.close()
 return text

filename = "Flickr8k.token.txt"
doc = load(filename)

In [ ]:
def create_desc(description):
    captions={}
    for line in doc.split('\n'):
        if len(line)<2:
            continue
        letters=line.split()
        img_id=letters[0]
        desc=letters[1:]
        img_id = img_id.split('.')[0]
        desc = ' '.join(desc)
        if img_id not in captions:
            captions[img_id]=[]
        captions[img_id].append(desc)
    return captions


In [ ]:
descriptions=create_desc(doc)
len(descriptions.keys())

8092

In [ ]:
def clean_text(desc):

    table = str.maketrans('', '', string.punctuation)
    for key, text in desc.items():
        for i in range(len(text)):
            text1 = text[i].split()
            text1 = [word.lower() for word in text1]


            text1 = [word.translate(table) for word in text1]


            text1 = [word for word in text1 if len(word) > 1]
            text1 = [word for word in text1 if word.isalpha()]
            text[i] = ' '.join(text1)
clean_text(descriptions)


In [ ]:
def to_vocabulary(descriptions):
    all_desc = set()
    for key in descriptions.keys():
        for d in descriptions[key]:
            all_desc.update(d.split())
    return all_desc

In [ ]:
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

Vocabulary Size: 8763


In [ ]:
def save(descriptions, file_path):
    saved = []
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            saved.append(key + ' ' + desc)
    data = '\n'.join(saved)


    with open(file_path, 'w') as file:
        file.write(data)


In [ ]:
save(descriptions, "C:\\ML-DATA\\Image_Captioning\\descriptions.txt")


In [ ]:
sample =open("C:\ML-DATA\Image_Captioning\descriptions.txt", 'r')
sample.read(200)

'1000268201_693b08cb0e child in pink dress is climbing up set of stairs in an entry way\n1000268201_693b08cb0e girl going into wooden building\n1000268201_693b08cb0e little girl climbing into wooden play'

## ***Load Data***

In [ ]:
def load_doc(filename):

 file = open(filename, 'r')
 text = file.read()
 file.close()
 return text


def load_set(filename):
 doc = load_doc(filename)
 dataset = list()

 for line in doc.split('\n'):

  if len(line) < 1:
   continue

  identifier = line.split('.')[0]
  dataset.append(identifier)
 return set(dataset)

In [ ]:
def load_clean_descriptions(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()

    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]

        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = list()

            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)

    return descriptions


In [ ]:
def load_photo_features(filename, dataset):
    try:
        with open(filename, 'rb') as f:
            all_features = pickle.load(f)
        features = {}
        for k in dataset:
            if k in all_features:
                features[k] = all_features[k]
            else:
                print(f"Warning: Key '{k}' not found in features file")

        return features
    except Exception as e:
        print(f"Error loading features: {str(e)}")
        return None


In [ ]:
filename = "Flickr_8k.trainImages.txt"
train = load_set(filename)
print('Dataset: %d' % len(train))
train_descriptions = load_clean_descriptions(r"C:\ML-DATA\Image_Captioning\descriptions.txt", train)
# descriptions
print('Descriptions: train=%d' % len(train_descriptions))

# photo features
train_features = load_photo_features('features.pkl', train)
if train_features is not None:
    print('Photos: train=%d' % len(train_features))
else:
    print("Failed to load photo features")

Dataset: 6000
Descriptions: train=6000
Photos: train=6000


## ***Encoding Text***

In [ ]:

def to_lines(descriptions):
 all_desc = list()
 for key in descriptions.keys():
  [all_desc.append(d) for d in descriptions[key]]
 return all_desc

def create_tokenizer(descriptions):
    lines=to_lines(descriptions)
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 7579


## ***Creating Input-Output pairs***

In [ ]:


def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
    X1, X2, y = list(), list(), list()
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)


In [ ]:
def max_length(descriptions):
  lines = to_lines(descriptions)
  return max(len(d.split()) for d in lines)

## ***Defining Model***

In [ ]:
def define_model(vocab_size, max_length):
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    print(model.summary())
    return model


In [ ]:
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
    while True:
        for key, desc_list in descriptions.items():
            photo = photos[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
            yield (np.array(in_img), np.array(in_seq)), np.array(out_word)


In [ ]:
filename = "Flickr_8k.trainImages.txt"
train = load_set(filename)
print('Dataset: %d' % len(train))
train_descriptions = load_clean_descriptions(r"C:\ML-DATA\Image_Captioning\descriptions.txt", train)
print('Descriptions: train=%d' % len(train_descriptions))
train_features = load_photo_features('features.pkl', train)
print('Photos: train=%d' % len(train_features))
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)


Dataset: 6000
Descriptions: train=6000
Photos: train=6000
Vocabulary Size: 7579
Description Length: 34


In [ ]:
generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)


(47, 4096)
(47, 34)
(47, 7579)


## ***Training Model***

In [ ]:
model = define_model(vocab_size, max_length)

epochs = 4
steps = len(train_descriptions)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('model_' + str(i) + '.h5')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 34)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 4096)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 34, 256)              1940224   ['input_3[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 4096)                 0         ['input_2[0][0]']             
                                                                                            

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6000/6000 [==============================] - 531s 89ms/step - loss: 3.5192


In [ ]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

In [ ]:
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = [],[]
    for key, desc_list in descriptions.items():
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())

    print('BLEU-1:', corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2:',corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3:',corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4:',corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [ ]:
filename = 'Flickr_8k.testImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))

test_descriptions = load_clean_descriptions(r"C:\ML-DATA\Image_Captioning\descriptions.txt", test)
print('Descriptions: test=%d' % len(test_descriptions))

test_features = load_photo_features('features.pkl', test)
print('Photos: test=%d' % len(test_features))
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 1000
Descriptions: test=1000
Photos: test=1000
BLEU-1: 0.5448866498740554
BLEU-2: 0.29916856376955797
BLEU-3: 0.20154242123012905
BLEU-4: 0.0916960212061892
